## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意（ここでは `ORDER BY day` を付けますが省略可）
* `NOT IN` は使用せず、今回はそもそも不要
* **判定は user_id 基準**（1 日に 1 回以上アクティビティがあれば「アクティブ」）
* Activity テーブルは **重複行あり** なので、**日 + user_id でユニークにしてから件数カウント**する 

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  2019-07-27 を終点とする直近 30 日間について、日別アクティブユーザー数（1 日に 1 回以上アクティビティがあった user 数）を求める。アクティビティ種別は `('open_session', 'end_session', 'scroll_down', 'send_message')` のどれであっても有効とみなす。ユーザー数が 0 人の日は結果に含めない。

* `{{TABLES_OR_SCHEMAS}}`

  ```text
  Table: Activity
  +---------------+---------+
  | Column Name   | Type    |
  +---------------+---------+
  | user_id       | int     |
  | session_id    | int     |
  | activity_date | date    |
  | activity_type | enum    -- 'open_session', 'end_session', 'scroll_down', 'send_message'
  +---------------+---------+
  -- 1 session_id は必ず 1 user_id に属する
  -- 行は重複している可能性がある
  ```

* `{{OUTPUT_COLUMNS_AND_RULES}}`

  ```text
  出力:
  +------------+--------------+
  | day        | active_users |
  +------------+--------------+
  | DATE       | INT          |
  +------------+--------------+

  ルール:
  - 対象期間は 2019-06-28 ～ 2019-07-27（両端を含む）
    └ 2019-07-27 を含む 30 日間なので 27 日 - 29 日 = 28 日
  - ある day に 1 回以上アクティビティを行った user_id を「その日のアクティブユーザー」と数える
  - 日別のアクティブユーザー数を集計する
  - アクティブユーザーが 0 人の日は結果から除外
  - 結果の並び順は任意（ここでは day 昇順）
  ```

---

## 2) 最適解（単一クエリ）

> 今回はウィンドウ関数は不要なので、**CTE でユニーク化 → GROUP BY 集計**のシンプル構成にします。

```sql
WITH uniq_activity AS (
  SELECT DISTINCT
    user_id,
    activity_date
  FROM Activity
  WHERE activity_date BETWEEN DATE '2019-06-28' AND DATE '2019-07-27'
)
SELECT
  activity_date AS day,
  COUNT(*)      AS active_users
FROM uniq_activity
GROUP BY activity_date
ORDER BY day;

Runtime 351 ms
Beats 67.57%

```

### ポイント

* `uniq_activity` で

  * 期間フィルタ: `activity_date BETWEEN DATE '2019-06-28' AND DATE '2019-07-27'`
  * `user_id, activity_date` を `DISTINCT` して同じユーザー・同じ日に複数行あっても 1 回と数える準備
* メインクエリで

  * `GROUP BY activity_date` し `COUNT(*)` でその日のアクティブユーザー数を算出
* 0 ユーザーの日はそもそも行が存在しないので結果に出てこない（要件どおり）

`COUNT(DISTINCT user_id)` を直接使ってもよいですが、重複除去と期間フィルタを明示したほうが読みやすいため、CTE で分けています。

---

## 3) 要点解説

1. **30 日間の計算方法**

   * 終端: `2019-07-27`
   * そこから 29 日前: `2019-06-28`
   * よって `BETWEEN DATE '2019-06-28' AND DATE '2019-07-27'` で 30 日間がカバーされる

2. **「アクティブユーザー」の定義を SQL に落とす**

   * 1 日に 1 回以上アクティビティがあればよい
     → 「日 + user_id の組み合わせ」を 1 ユニットとして数えればよい
   * そのために `SELECT DISTINCT user_id, activity_date` で重複を除く

3. **重複行対策**

   * 元テーブルに全く同一の行が複数あっても、

     * CTE 内で `DISTINCT(user_id, activity_date)` することで
     * 集計時のカウントに影響が出ないようにしている

4. **0 ユーザー日の扱い**

   * Activity に 1 行もない日付は CTE にも現れない
   * GROUP BY 結果にも出てこないため、自動的に「0 ユーザーの日は結果に含めない」挙動になる

5. **インデックス設計（パフォーマンス）**

   * 実務では次の複合インデックスが有効

     ```sql
     CREATE INDEX idx_activity_date_user
       ON Activity (activity_date, user_id);
     ```
   * これにより

     * 範囲条件 `activity_date BETWEEN ...` の絞り込み
     * `user_id, activity_date` の `DISTINCT` / `GROUP BY`
       の両方が効率化される

---

## 4) 計算量（概算）

* Activity テーブルの行数を **N** としたとき

1. **CTE 内部 (`DISTINCT` + 範囲フィルタ)**

   * 範囲フィルタ: おおむね **O(N)**（インデックスがあれば実質 O(N_range)）
   * `DISTINCT(user_id, activity_date)`:

     * ハッシュ集約 or ソートベースで **O(N_range)** ～ **O(N_range log N_range)**

2. **外側の `GROUP BY`**

   * すでに `(user_id, activity_date)` はユニークなので、
   * 日数を D とすると **O(D)** 〜 **O(D log D)** 程度（D は最大 30 日）

⇒ 全体としては **O(N_range log N_range)** が支配的
（`COUNT(DISTINCT user_id)` を直接使う場合も同程度）

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Activity テーブル] --> B["日付範囲でフィルタ 2019-06-28〜2019-07-27"]
  B --> C["日 + user_id で DISTINCT (uniq_activity CTE)"]
  C --> D["activity_date ごとに GROUP BY COUNT(*)"]
  D --> E[出力: day, active_users]
```

このクエリをそのまま PostgreSQL 16.6+ 環境に流し込めば、問題の期待する結果形式で日別アクティブユーザー数を取得できます。
結論から言うと、

* **351ms / 上位 67% なら十分速い部類**
* LeetCode 環境（インデックス作れない・データ量少なめ）では **劇的な改善はほぼ出ません**
* それでも「やれること」は 2〜3 個あります

という感じです。

---

## 1. クエリのシンプル化（1 段階集約）

前回お渡ししたのは

```sql
WITH uniq_activity AS (
  SELECT DISTINCT
    user_id,
    activity_date
  FROM Activity
  WHERE activity_date BETWEEN DATE '2019-06-28' AND DATE '2019-07-27'
)
SELECT
  activity_date AS day,
  COUNT(*)      AS active_users
FROM uniq_activity
GROUP BY activity_date
ORDER BY day;
```

でしたが、**LeetCode のような小さめデータ**であれば、
CTE を使わずに **1 回の GROUP BY + COUNT(DISTINCT)** にまとめた方が
プランナーが素直に最適化してくれることも多いです。

```sql
SELECT
  activity_date AS day,
  COUNT(DISTINCT user_id) AS active_users
FROM Activity
WHERE activity_date BETWEEN DATE '2019-06-28' AND DATE '2019-07-27'
GROUP BY activity_date;
-- ORDER BY は仕様上不要なら外してよい

Runtime 366 ms
Beats 48.24%

```

### なぜ速くなる可能性があるか

* CTE + DISTINCT + GROUP BY という **2 段階の集約** → 中間結果のマテリアライズが起きることがある
* 一方、`COUNT(DISTINCT user_id)` なら **1 回の GROUP BY** で完結
* 特に LeetCode ではオプティマイザ設定もそこまで凝ってないので、
  **素直な単一クエリの方が有利なことが多い**です

---

## 2. ORDER BY を外す（仕様 OK なら）

問題文が「順序は任意」と明言しているので、
**本当に順序不要**であれば `ORDER BY` を消すとソート処理がなくなり、その分は確実に速くなります。

```sql
SELECT
  activity_date AS day,
  COUNT(DISTINCT user_id) AS active_users
FROM Activity
WHERE activity_date BETWEEN DATE '2019-06-28' AND DATE '2019-07-27'
GROUP BY activity_date;
```

ソートは `O(D log D)`（D は日数）なので、
この問題では日数が最大 30 日と小さく、**体感としてはあまり変わりませんが**、
純粋なランタイムだけを見れば少し良くなる可能性があります。

---

## 3. 実務ならインデックスを貼る

LeetCode では出来ませんが、実務 PostgreSQL ならこうします：

```sql
CREATE INDEX idx_activity_date_user
  ON Activity (activity_date, user_id);
```

これで

* `activity_date BETWEEN ...` の範囲スキャン
* `COUNT(DISTINCT user_id)` の内部での group/hash

が両方とも効率化されます。

---

## 4. パフォーマンス評価の目安

* 上位 **67%** は「解として十分」「実務でも困らないレベル」です
* もし上位 90% を狙うなら

  * 上記のように **1 段階集約 + ORDER BY 削除** を試す
  * それでも改善が小さい／変わらないことも多い（データ量が小さいため）

なので、**アルゴリズム／クエリ設計としては合格点以上**で、
あとは「CTE を減らして素直な集計に寄せる」くらいの微調整フェーズ、というイメージです。

---

もし「実務で数千万〜数億行ある場合はどうチューニングする？」みたいな視点も気になるようなら、
その前提でインデックス設計や EXPLAIN の読み方まで掘っていきましょう 💪
